In [1]:
import tensorflow as tf
import numpy as np
import mlp.tf_utils as utils
from mlp.data_providers import AugmentedCIFAR10DataProvider, AugmentedCIFAR100DataProvider, CIFAR100DataProvider, CIFAR10DataProvider
from mlp.image_transforms import random_flip, random_crop, center_crop, random_flip_small
from mlp.Conv_models import ConvModel, TwoTaskConvModel
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

seed = 12345 
rng = np.random.RandomState(seed)

In [2]:
a = tf.Variable(tf.truncated_normal([1, 1], stddev=0.2))
b = tf.Variable(np.array([1,2]))
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(a))
    n = a.assign(tf.truncated_normal([1, 1], stddev=0.2))
    sess.run(init)
    print(sess.run(b))
    print(sess.run(a))
    print(sess.run(a.value()))
    print(sess.run(a.initialized_value()))
    print(sess.run(a.initialized_value()))
    print(sess.run(a.initial_value))
    print(type(sess.run(a.initial_value)))

[[-0.30379316]]
[1 2]
[[ 0.00297204]]
[[ 0.00297204]]
[[ 0.04974794]]
[[ 0.10570712]]
[[ 0.25545815]]
<type 'numpy.ndarray'>


In [3]:
layer_dims = [3, 64, 128, 128, 128, _]
batch_size = 128

bifurcation_point = 2
joint_loss = False
max_pools = [True, True, False, True]
lrns = []
lrn_alphas = []
batch_norms = [True, True, True, True]
l1 = False
l2 = 5e-4
learning_rates = [1e-3]  
lrn_schedule = []
optimizer_params = [0.9, 0.999]  
activation = tf.nn.relu
dropout = False
epochs = 30
error = 'soft_max_cross_entropy'
image_size = 32  
optimiser = "GD"  
logs_dir = "tf-log"

In [4]:
mtl_models = []
b_models = []

for i, ratio in enumerate([1.]):
    
    ########## SORT DATA OUT ###########################
    main_train_data = CIFAR100DataProvider(which_set='train',
                                                    batch_size=batch_size, shuffle_order=False)
    main_valid_data = CIFAR100DataProvider(which_set='valid', batch_size=batch_size, shuffle_order=False)

    aux_train_data = CIFAR100DataProvider(which_set='train',
                                                   batch_size=batch_size, shuffle_order=False, use_coarse_targets=False)
    aux_valid_data = CIFAR100DataProvider(which_set='valid', batch_size=batch_size, 
                                          use_coarse_targets=False, shuffle_order=False)
    
    m_t_idx = utils.sample_data(main_train_data, ratio)
    m_v_idx = utils.sample_data(main_valid_data, ratio)
    _1 = utils.sample_data(aux_train_data, ratio, m_t_idx)
    _2 = utils.sample_data(aux_valid_data, ratio, m_v_idx)
    
    train_data = CIFAR100DataProvider(which_set='train', batch_size=batch_size)
    valid_data = CIFAR100DataProvider(which_set='valid', batch_size=batch_size)
    
    # update baseline data providers to have same samples 
    _3 = utils.sample_data(train_data, ratio, m_t_idx)
    _4 = utils.sample_data(valid_data, ratio, m_v_idx)
    ######################################################
    
    
    ##################### GET BASELINE VARIABLES #######################
    tf.reset_default_graph()
    model_name = 'baseline_with_' + str(ratio) + "_of_the_data"
    print("MODEL:- " + model_name)
    
    model_b = ConvModel(conv_layer_dims=layer_dims, train_data=train_data, valid_data=valid_data,
                 batch_size=batch_size, learning_rates=learning_rates, learning_rate_schedule=[],
                 optimiser=optimiser, L1=l1, L2=l2,
                 max_pools=max_pools, lin_response_norms=lrns, lin_response_alphas=lrn_alphas, batch_norms=batch_norms,
                 optimiser_params=optimizer_params, activation=activation, dropout=dropout,
                 epochs=epochs, error=error, input_image_size=image_size, name=model_name)
    model_b.create_network()
    model_b.initialize_network(logs_dir)
    model_b.train_model()
    b_models.append(model_b)
    ###################################################################
    
    ############### CAPTURE VARIABLES ########################
    var = {}
    with model_b.sess as sess:
        for v in model_b.train_variables:
            if "W_2" in v.op.name:
                var["W_2_main"] = sess.run(v)
                var["W_2_aux"] = sess.run(v) 
            elif "b_2" in v.op.name:
                var["b_2_main"] = sess.run(v)
                var["b_2_aux"] = sess.run(v)
            elif "W_3" in v.op.name:
                var["W_3_main"] = sess.run(v)
                var["W_3_aux"] = sess.run(v) 
            elif "b_3" in v.op.name:
                var["b_3_main"] = sess.run(v)
                var["b_3_aux"] = sess.run(v)
            elif "W_pred" in v.op.name:
                var["W_main_pred"] = sess.run(v)
                var["W_aux_pred"] = sess.run(v)
            elif "b_pred" in v.op.name:
                var["b_main_pred"] = sess.run(v)
                var["b_aux_pred"] = sess.run(v)
            
    model_name = 'sanity_check_' + str(ratio) + "_of_the_data"
    print("MODEL:- " + model_name)
    
    model = TwoTaskConvModel(conv_layer_dims=layer_dims, main_train_data=main_train_data, main_valid_data=main_valid_data,
                        aux_train_data=aux_train_data, aux_valid_data=aux_valid_data, L1=l1, L2=l2,
                         batch_size=batch_size, learning_rates=learning_rates, learning_rate_schedule=lrn_schedule,
                         optimiser=optimiser, max_pools=max_pools, lin_response_norms=lrns,
                         lin_response_alphas=lrn_alphas, batch_norms=batch_norms, bifurcation_point=bifurcation_point,
                         optimiser_params=optimizer_params, activation=activation, dropout=dropout,
                         epochs=epochs, error=error, input_image_size=image_size, name=model_name, joint_loss=joint_loss)
    
    model.create_network(var_dict=var)
    model.initialize_network(logs_dir)
    model.train_model()

    mtl_models.append(model)

MODEL:- baseline_with_1.0_of_the_data
Setting up model...
Initializing network...
Training model...
Epoch finished:  1
Epoch finished:  2
Epoch finished:  3
Epoch finished:  4
Epoch finished:  5
Epoch finished:  6
Epoch finished:  7
Epoch finished:  8
Epoch finished:  9
Epoch finished:  10
Epoch finished:  11
Epoch finished:  12
Epoch finished:  13
Epoch finished:  14
Epoch finished:  15
Epoch finished:  16
Epoch finished:  17
Epoch finished:  18
Epoch finished:  19
Epoch finished:  20
Epoch finished:  21
Epoch finished:  22
Epoch finished:  23
Epoch finished:  24
Epoch finished:  25
Epoch finished:  26
Epoch finished:  27
Epoch finished:  28
Epoch finished:  29
Epoch finished:  30
Time taken: 296.09553504
MODEL:- sanity_check_1.0_of_the_data
Setting up model...
W_2_main copied
W_2_aux copied
b_2_main copied
b_2_aux copied
W_3_main copied
W_3_aux copied
b_3_main copied
b_3_aux copied
Initializing network...
Training model...
Epoch finished:  1
Epoch finished:  2
Epoch finished:  3
Epoc

In [5]:
b = b_models[0]
for v in b.train_variables:
    print(v.op.name)
print( "  -----------   ")
m = mtl_models[0]
for v in m.train_variables:
    print(v.op.name)

baseline_with_1.0_of_the_data_conv_model/W_0
baseline_with_1.0_of_the_data_conv_model/b_0
baseline_with_1.0_of_the_data_conv_model/BatchNorm/beta
baseline_with_1.0_of_the_data_conv_model/W_1
baseline_with_1.0_of_the_data_conv_model/b_1
baseline_with_1.0_of_the_data_conv_model/BatchNorm_1/beta
baseline_with_1.0_of_the_data_conv_model/W_2
baseline_with_1.0_of_the_data_conv_model/b_2
baseline_with_1.0_of_the_data_conv_model/BatchNorm_2/beta
baseline_with_1.0_of_the_data_conv_model/W_3
baseline_with_1.0_of_the_data_conv_model/b_3
baseline_with_1.0_of_the_data_conv_model/BatchNorm_3/beta
baseline_with_1.0_of_the_data_conv_model/W_pred
baseline_with_1.0_of_the_data_conv_model/b_pred
  -----------   
baseline_with_1.0_of_the_data_conv_model/W_0
baseline_with_1.0_of_the_data_conv_model/b_0
baseline_with_1.0_of_the_data_conv_model/BatchNorm/beta
baseline_with_1.0_of_the_data_conv_model/W_1
baseline_with_1.0_of_the_data_conv_model/b_1
baseline_with_1.0_of_the_data_conv_model/BatchNorm_1/beta
bas